In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "Helvetica"
})

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()

from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC  
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.metrics import auc
from sklearn.metrics import roc_curve

In [2]:
tot_fit = pd.read_csv('./HI_ML/final_fit.csv', index_col=0)
tot_fit

,Filename,Success,Nchan,dof,chi^2,chi^2/dof,rms,A,dA,B_1,...,dX,W_50,dW_50,W_20,dW_20,F_peak,dF_peak,F_int,dF_int,Class
0,0003+380_ak18.DAT,0,42,37,22.08120,0.596788,0.01,7.033000e-02,4.490010e-02,0.158193,...,0.000000e+00,0.00000,0.000000e+00,0.00000,0.000000e+00,0.056207,5.420480e-03,0.000000,0.000000e+00,0
1,0035+22_omd+17.DAT,0,57,49,3.19013,0.065105,0.01,7.603450e-03,1.642290e-01,3.618170,...,5.104230e+03,169.00500,1.372020e+04,183.26100,1.044720e+04,0.020974,2.964040e-01,2.507320,1.558770e+02,0
2,0105-008_gmrt.freq,1,171,166,11.97330,0.072128,0.01,2.438270e-01,3.728630e+00,0.139187,...,1.250210e+00,15.33200,1.821070e+00,23.94890,2.427310e+00,0.059377,5.261230e-03,0.986714,9.323590e-02,0
3,0941-08_omd+17.DAT,2,63,58,13.72410,0.236623,0.01,1.816270e-02,5.183430e-03,0.375388,...,1.694480e+17,271.15100,1.903000e+17,316.23400,2.873760e+17,0.018163,9.322750e+04,4.928510,5.865600e+15,0
4,101301.DAT,0,154,149,3.49873,0.023481,0.01,5.352310e-03,1.697720e-03,0.050057,...,2.363250e+01,201.07300,5.071530e+01,243.04300,6.576720e+01,0.005352,1.694540e-03,1.076170,2.963070e-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,SPEC_J1639.DAT,0,512,504,6577.90000,13.051400,0.01,4.809480e-01,2.592260e-01,0.210122,...,5.206090e-02,23.00760,1.183090e-01,30.30620,2.114130e-01,0.661494,4.192530e-03,15.045100,6.214660e-02,1
40,SPEC_J2219+0229.DAT,1,1117,1109,5530.31000,4.986750,0.01,2.891930e-01,1.366910e+01,0.222174,...,4.426460e-01,11.09360,3.617770e-01,36.07690,8.071830e-01,0.250818,6.521780e-03,3.945680,1.054660e-01,1
41,1331+170_arecibo.freq,1,247,242,112.78600,0.466058,0.01,5.307520e+07,7.146710e+10,0.010981,...,2.015020e+00,24.36920,6.327290e+00,37.92810,6.963300e+00,0.034750,6.056060e-03,0.920237,2.582000e-01,1
42,SPEC_J0041S.DAT,0,483,475,489.00700,1.029490,0.01,6.564260e-02,1.071500e-01,0.519419,...,7.477110e-01,8.35918,1.629950e+00,12.39470,2.113430e+00,0.057690,8.816000e-03,0.501437,6.219430e-02,1


# Decision Tree

## Non Redshifted Parameters

In [3]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps_dt = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', DecisionTreeClassifier(criterion='gini', max_depth=4, max_features=None, min_samples_leaf=4, min_samples_split=2 , splitter = 'best', random_state=0))]#('model', DecisionTreeClassifier(criterion='gini', max_depth=1, max_features=None, min_samples_leaf=1, min_samples_split=2, splitter='best', random_state=0))]
pipeline_dt = Pipeline(steps=steps_dt)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline_dt, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.862
accuracy is 0.828
average_precision is 0.772
recall is 0.79
neg_log_loss is -3.162


In [4]:
## Previous best fit

# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps_dt = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', DecisionTreeClassifier(criterion='gini', max_depth=4, max_features=None, min_samples_leaf=4, min_samples_split=2 , splitter = 'best', random_state=0))]
pipeline_dt = Pipeline(steps=steps_dt)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline_dt, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.862
accuracy is 0.828
average_precision is 0.772
recall is 0.79
neg_log_loss is -3.162


## Redshifted Parameters

In [5]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values 
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', DecisionTreeClassifier(criterion='gini', max_depth=1, max_features=None, min_samples_leaf=1, min_samples_split=2, splitter='best', random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.871
accuracy is 0.87
average_precision is 0.766
recall is 0.874
neg_log_loss is -0.406


In [6]:
## Previous best fit
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values 
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', DecisionTreeClassifier(criterion='gini', max_depth=6, max_features='sqrt', min_samples_leaf=2, min_samples_split=3 , splitter = 'random', random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.903
accuracy is 0.877
average_precision is 0.83
recall is 0.884
neg_log_loss is -1.455


# Random Forest

## Non Redshifted Parameters

In [7]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', RandomForestClassifier(n_estimators=1, max_depth=1, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, bootstrap=True, random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} for SMOTE Model is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc for SMOTE Model is 0.608
accuracy for SMOTE Model is 0.677
average_precision for SMOTE Model is 0.495
recall for SMOTE Model is 0.345
neg_log_loss for SMOTE Model is -0.711


In [8]:
## Previous best fit
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', RandomForestClassifier(n_estimators=9, max_depth=7, max_features=None, min_samples_leaf=4, min_samples_split=2, bootstrap=True, random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} for SMOTE Model is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc for SMOTE Model is 0.91
accuracy for SMOTE Model is 0.858
average_precision for SMOTE Model is 0.868
recall for SMOTE Model is 0.854
neg_log_loss for SMOTE Model is -0.799


## Redshifted Parameters

In [9]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', RandomForestClassifier(n_estimators=1, max_depth=1, max_features='sqrt', min_samples_leaf=1, min_samples_split=2, bootstrap=True, random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.844
accuracy is 0.839
average_precision is 0.719
recall is 0.863
neg_log_loss is -0.522


In [10]:
## Previous best fit
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', RandomForestClassifier(n_estimators=13, max_depth=6, max_features='sqrt', min_samples_leaf=1, min_samples_split=7, bootstrap=False, random_state=0))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.953
accuracy is 0.905
average_precision is 0.935
recall is 0.873
neg_log_loss is -0.384


# Logistic Regression

## Non Redshifted Parameters

In [11]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('scaler', StandardScaler()), ('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', LogisticRegressionCV(max_iter=100, Cs= 1, penalty='l1', random_state=0, solver='liblinear', class_weight=None, fit_intercept=True, n_jobs=-1))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.5
accuracy is 0.63
average_precision is 0.37
recall is 0.0
neg_log_loss is -0.693


In [ ]:
## Previous best fit
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', LogisticRegressionCV(max_iter=100, Cs= 100, fit_intercept=True, penalty='l1', random_state=0, solver='saga', n_jobs=4, class_weight= None))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

append_list = []
for i in score_list:
    append_list.append(round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3))
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3)))

In [14]:
j = 0
for i in append_list:
    print('{} for is {}'.format(score_list[j], i))
    j+=1

roc_auc for SVM SMOTE Model is 0.453
accuracy for SVM SMOTE Model is 0.507
average_precision for SVM SMOTE Model is 0.469
recall for SVM SMOTE Model is 0.438
neg_log_loss for SVM SMOTE Model is -0.717


# Redshifted Parameters

In [15]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', LogisticRegressionCV(max_iter=100, Cs= 1, penalty='l1', random_state=0, solver='liblinear', class_weight=None, fit_intercept=True, n_jobs=-1))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

for i in score_list:
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=-1)), 3)))

roc_auc is 0.505
accuracy is 0.582
average_precision is 0.488
recall is 0.302
neg_log_loss is -0.717


In [ ]:
## Previous best fit
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', LogisticRegressionCV(max_iter=100, Cs= 100, fit_intercept=True, penalty='l1', random_state=0, solver='saga', n_jobs=4, class_weight= None))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

append_list = []
for i in score_list:
    append_list.append(round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3))
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3)))

In [17]:
j = 0
for i in append_list:
    print('{} is {}'.format(score_list[j], i))
    j+=1

roc_auc is 0.634
accuracy is 0.671
average_precision is 0.616
recall is 0.342
neg_log_loss is -0.717


# Support Vector Machine

## Non Redshifted Parameters

In [ ]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 21, 29, 31]].values #excluding 19, 23, 25, 27 columns
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('scaler', StandardScaler()), ('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', SVC(C = 0.1, kernel='linear', coef0=0, degree=2, gamma='scale', max_iter=100, shrinking=True, tol=0.001, random_state=0, probability=True))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

append_list = []
for i in score_list:
    append_list.append(round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3))
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3)))

In [19]:
j = 0
for i in append_list:
    print('{} is {}'.format(score_list[j], i))
    j+=1

roc_auc is 0.833
accuracy is 0.682
average_precision is 0.772
recall is 0.911
neg_log_loss is -0.755


## Redshifted Parameters

In [ ]:
# input
X = tot_fit.iloc[:, [7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31]].values
# output
y = tot_fit.iloc[:, 33].values.astype('int')

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=100, random_state=0)

steps = [('over', SMOTE(sampling_strategy=1, random_state=0)), ('model', SVC(C = 0.1, kernel='linear', coef0=0, degree=2, gamma='scale', max_iter=100, shrinking=True, tol=0.001, random_state=0, probability=True))]
pipeline = Pipeline(steps=steps)

score_list = ['roc_auc', 'accuracy', 'average_precision', 'recall', 'neg_log_loss']

append_list = []
for i in score_list:
    append_list.append(round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3))
    print('{} is {}'.format(i, round(np.mean(cross_val_score(pipeline, X, y, scoring=i, cv=cv, n_jobs=4)),3)))

In [34]:
j = 0
for i in append_list:
    print('{} is {}'.format(score_list[j], i))
    j+=1

roc_auc is 0.567
accuracy is 0.427
average_precision is 0.558
recall is 0.793
neg_log_loss is -0.746
